In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 131331 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.6-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.6-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.6-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

In [ ]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [3]:
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag

platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\10/'    
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'
version='1.0.0'
torch_url=f"http://download.pytorch.org/whl/{accelerator}/torch-{version}-{platform}-linux_x86_64.whl"

!pip install -U {torch_url} torchvision

     |████████████████████████████████| 753.6MB 91.4MB/s 
Requirement already up-to-date: torchvision in /usr/local/lib/python3.6/dist-packages (0.3.0)
ERROR: torchvision 0.3.0 has requirement torch>=1.1.0, but you'll have torch 1.0.0 which is incompatible.
  Found existing installation: torch 1.1.0
    Uninstalling torch-1.1.0:
      Successfully uninstalled torch-1.1.0


In [4]:
!ls drive/Colab/datagrad

baselines.ipynb  data  model


In [5]:
!pip install fasttext
import fasttext.FastText as fasttext

     |████████████████████████████████| 61kB 2.8MB/s eta 0:00:011
  Stored in directory: /root/.cache/pip/wheels/9f/f0/04/caa82c912aee89ce76358ff954f3f0729b7577c8ff23a292e3
Successfully built fasttext


# Init

In [ ]:
import torch.nn.functional as F
import torch
from torch import nn, optim
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support
from tqdm import tqdm

import random
import os
import pickle
import copy

local = False
if local:
    DATAPATH = '/Users/alfonso/workplace/datagrad/Data'
    MODELPATH = '/Users/alfonso/workplace/datagrad/Model'
    
else:
    PATH = '/content/drive/'
    DATAPATH = os.path.join(PATH,'Colab/datagrad/data/')
    MODELPATH = os.path.join(PATH,'Colab/datagrad/model/')

START_TAG = "<START>"
STOP_TAG = "<STOP>"
tag_to_ix = {"a": 0, "b": 1, "c": 2, "o": 3,
             START_TAG: 4, STOP_TAG: 5}
ix_to_tag = {tp[1]: tp[0] for tp in tag_to_ix.items()}
ix_to_tag[5] = "o"
ix_to_tag[4] = "o"

# Functions

In [ ]:
def load_pretrained(name, word_to_ix, embed_size=128):
    model = fasttext.load_model(os.path.join(
        MODELPATH, '{}.bin'.format(name)))
    embedding_matrix = np.zeros((len(word_to_ix), embed_size))
    for w, ix in word_to_ix.items():
        embedding_matrix[ix] = model[w]
    return embedding_matrix

def fromPickle(path):
    var = None
    with open(path, 'rb') as f:
        var = pickle.load(f)
    return var

def load():
    trn_X = fromPickle(os.path.join(
        DATAPATH, 'trn_X_token.pickle'))#[:10]
    trn_y = fromPickle(os.path.join(
        DATAPATH, 'trn_y_token.pickle'))#[:10]
    tst = fromPickle(os.path.join(
        DATAPATH, 'tst_X_token.pickle'))#[:10]
    word_to_ix = fromPickle(os.path.join(
        DATAPATH, 'word_to_ix.pickle'))
    return trn_X, trn_y, tst, word_to_ix

def savemodel(model, name):
    torch.save(model.state_dict(), os.path.join(MODELPATH, name))

    
def _evalinfo(y_preds, y_trues):
    size = len(y_preds)
    assert len(y_preds) == len(y_trues)
    stat = {'a': list([0,0]), 'b': list([0,0]), 'c': list([0,0])}
    tags = ['a', 'b', 'c']
    prec = {'a': list(), 'b': list(), 'c': list()}
    recl = {'a': list(), 'b': list(), 'c': list()}
    f1 = {'a': 0, 'b': 0, 'c': 0}
    for j in range(len(y_preds)):
        y_pred, y_true = y_preds[j], y_trues[j]
        prestat, recstat = copy.copy(stat), copy.copy(stat)
        for i in range(len(y_pred)):
            t = ix_to_tag[int(y_true[i])]
            p = ix_to_tag[int(y_pred[i])]
            if p in tags: # predict result
                prestat[p][1] += 1
            if p in tags and t in tags and p == t: # predict correct
                prestat[p][0] += 1
                recstat[t][0] += 1
            if t in tags: # true result
                recstat[t][1] += 1
        for x in tags:
            if recstat[x][1] != 0:
                recl[x].append(recstat[x][0] /recstat[x][1])
            if prestat[x][1] != 0:
                prec[x].append(prestat[x][0] / prestat[x][1])
    for x in tags:
        prec[x] = 0 if len(prec[x]) == 0 else sum(prec[x])/len(prec[x])
        recl[x] = 0 if len(recl[x]) == 0 else sum(recl[x])/len(recl[x])
        f1[x] = (2 * prec[x] * recl[x]) / (prec[x] + recl[x] + 1e-8)

    for x in tags:
        print('TAG {} \t prec {:.4f} \t recl {:.4f} \t f1 {:.4f}'.format(
            x, prec[x], recl[x], f1[x]))

    print('AVG prec {:.4f} \t recl {:.4f} \t f1 {:.4f}'.format(
        sum(prec[x] for x in tags) / 3,
        sum(recl[x] for x in tags) / 3,
        sum(f1[x] for x in tags) / 3))
    
def evalinfo(y_preds, y_trues):
    tags = ['a', 'b', 'c']
    sgl_stat = np.zeros((3, 3))
    all_stat = np.zeros((3,))
    size = len(y_preds)
    for idx in range(size):
        y_pred, y_true = y_preds[idx], y_trues[idx]
        t = [ix_to_tag[int(ix)] for ix in y_true]
        p = [ix_to_tag[int(ix)] for ix in y_pred]
        tp = precision_recall_fscore_support(t, p, labels=tags)
        avgtp = precision_recall_fscore_support(
            t, p, labels=tags, average='macro')

        for i in range(3):  # p,r,f
            for j in range(3):  # a, b, c
                sgl_stat[i][j] += tp[i][j] / size

        for i in range(3):
            all_stat[i] += avgtp[i] / size
    for i, x in enumerate(tags):
        print('TAG {} \t prec {:.4f} \t recl {:.4f} \t f1 {:.4f}'.format(
            x, sgl_stat[0][i], sgl_stat[1][i], sgl_stat[2][i]))
    print('AVG prec {:.4f} \t recl {:.4f} \t f1 {:.4f}'.format(
        all_stat[0], all_stat[1], all_stat[2]))
    print()
          
def argmax(vec):
    _, idx = torch.max(vec, 1)
    return idx.item()

def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

def init_embedding(embedding):
    bias = np.sqrt(3.0 / embedding.embedding_dim)
    nn.init.uniform_(embedding.weight, -bias, bias)

def init_linear(linear):
    bias = np.sqrt(6.0 / (linear.weight.size(0) +
                          linear.weight.size(1)))
    nn.init.uniform_(linear.weight, -bias, bias)
    if linear.bias is not None:
        linear.bias.data.zero_()

def init_rnn(input_rnn, rnn='lstm'):
    part_num = 4 if rnn == 'lstm' else 3
    for ind in range(0, input_rnn.num_layers):
        weight = eval('input_rnn.weight_ih_l' + str(ind))
        hid_size = weight.size(0) // part_num
        for i in range(part_num):
            nn.init.xavier_normal_(weight[hid_size * i:hid_size * (i + 1), :])
        weight = eval('input_rnn.weight_hh_l' + str(ind))
        for i in range(part_num):
            nn.init.xavier_normal_(weight[hid_size * i:hid_size * (i + 1), :])
    if input_rnn.bidirectional:
        for ind in range(0, input_rnn.num_layers):
            weight = eval('input_rnn.weight_ih_l' + str(ind) + '_reverse')
            for i in range(part_num):
                nn.init.xavier_normal_(
                    weight[hid_size * i:hid_size * (i + 1), :])
            weight = eval('input_rnn.weight_hh_l' + str(ind) + '_reverse')
            for i in range(part_num):
                nn.init.xavier_normal_(
                    weight[hid_size * i:hid_size * (i + 1), :])
    if input_rnn.bias:
        for ind in range(0, input_rnn.num_layers):
            bias = eval('input_rnn.bias_ih_l' + str(ind))
            bias.data.zero_()
            bias.data[input_rnn.hidden_size: 2 * input_rnn.hidden_size] = 1
            bias = eval('input_rnn.bias_hh_l' + str(ind))
            bias.data.zero_()
            bias.data[input_rnn.hidden_size: 2 * input_rnn.hidden_size] = 1
        if input_rnn.bidirectional:
            for ind in range(0, input_rnn.num_layers):
                bias = eval('input_rnn.bias_ih_l' + str(ind) + '_reverse')
                bias.data.zero_()
                bias.data[input_rnn.hidden_size: 2 *
                          input_rnn.hidden_size] = 1
                bias = eval('input_rnn.bias_hh_l' + str(ind) + '_reverse')
                bias.data.zero_()
                bias.data[input_rnn.hidden_size: 2 *
                          input_rnn.hidden_size] = 1

def process(trn_ixs, vld_ixs, model, optimizer, device, epoch_num=2, mname='',no=''):
    print('No {} ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^'.format(no))
    for epoch in range(epoch_num):
        model.train()
        model, avg_loss = train(trn_ixs, model, optimizer, device)
        model.eval()
        vld_loss, y_preds, y_trues = evaluate(vld_ixs, model, device)
        print('Epoch {}/{} \t avg_loss {:.4f} \t vld_loss {:.4f} \t'.format(
            epoch + 1, epoch_num, avg_loss, vld_loss))
        evalinfo(y_preds, y_trues)
    savemodel(model, '{}_{}.pytorch'.format(mname,no))

# Model

## Focal Loss

In [ ]:
class FocalLoss(nn.Module):
    def __init__(self, num_class, alpha=None, gamma=2, balance_index=-1, smooth=None, size_average=True, device='cpu'):
        super(FocalLoss, self).__init__()
        self.num_class = num_class
        self.alpha = alpha
        self.gamma = gamma
        self.smooth = smooth
        self.size_average = size_average
        if self.alpha is None:
            self.alpha = torch.ones(self.num_class, 1)
        elif isinstance(self.alpha, (list, np.ndarray)):
            assert len(self.alpha) == self.num_class
            self.alpha = torch.FloatTensor(alpha).view(self.num_class, 1)
            self.alpha = self.alpha / self.alpha.sum()
        elif isinstance(self.alpha, float):
            alpha = torch.ones(self.num_class, 1)
            alpha = alpha * (1 - self.alpha)
            alpha[balance_index] = self.alpha
            self.alpha = alpha
        else:
            raise TypeError('Not support alpha type')
        if self.smooth is not None:
            if self.smooth < 0 or self.smooth > 1.0:
                raise ValueError('smooth value should be in [0,1]')

    def forward(self, input, target):
        logit = F.softmax(input, dim=1)
        if logit.dim() > 2:
            logit = logit.view(logit.size(0), logit.size(1), -1)
            logit = logit.permute(0, 2, 1).contiguous()
            logit = logit.view(-1, logit.size(-1))
        target = target.view(-1, 1)
        epsilon = 1e-10
        alpha = self.alpha
        if alpha.device != input.device:
            alpha = alpha.to(input.device)
        idx = target.cpu().long()
        one_hot_key = torch.FloatTensor(target.size(0), self.num_class).zero_()
        one_hot_key = one_hot_key.scatter_(1, idx, 1)
        if one_hot_key.device != logit.device:
            one_hot_key = one_hot_key.to(logit.device)
        if self.smooth:
            one_hot_key = torch.clamp(
                one_hot_key, self.smooth, 1.0 - self.smooth)
        pt = (one_hot_key * logit).sum(1) + epsilon
        logpt = pt.log()
        alpha = alpha[idx]
        loss = -1 * alpha * torch.pow((1 - pt), self.gamma) * logpt

        if self.size_average:
            loss = loss.mean()
        else:
            loss = loss.sum()
        return loss

## Init Functions

In [ ]:
class InitFns(object):

    def __init__(self, emb_fn=nn.init.uniform_, lnr_fn=nn.init.uniform_, nn_fn=nn.init.xavier_normal_):

        self.emb_fn = emb_fn
        self.lnr_fn = lnr_fn
        self.nn_fn = nn_fn

    def init_embedding(self, embedding):
        self.emb_fn(embedding.weight)

    def init_linear(self, linear):
        self.nn_fn(linear.weight)
        if linear.bias is not None:
            linear.bias.data.zero_()

    def init_rnn(self, input_rnn, rnn='lstm'):
        part_num = 4 if rnn == 'lstm' else 3
        for ind in range(0, input_rnn.num_layers):
            weight = eval('input_rnn.weight_ih_l' + str(ind))
            hid_size = weight.size(0) // part_num
            for i in range(part_num):
                self.nn_fn(
                    weight[hid_size * i:hid_size * (i + 1), :])
            weight = eval('input_rnn.weight_hh_l' + str(ind))
            for i in range(part_num):
                self.nn_fn(
                    weight[hid_size * i:hid_size * (i + 1), :])
        if input_rnn.bidirectional:
            for ind in range(0, input_rnn.num_layers):
                weight = eval('input_rnn.weight_ih_l' + str(ind) + '_reverse')
                for i in range(part_num):
                    self.nn_fn(
                        weight[hid_size * i:hid_size * (i + 1), :])
                weight = eval('input_rnn.weight_hh_l' + str(ind) + '_reverse')
                for i in range(part_num):
                    self.nn_fn(
                        weight[hid_size * i:hid_size * (i + 1), :])
        if input_rnn.bias:
            for ind in range(0, input_rnn.num_layers):
                bias = eval('input_rnn.bias_ih_l' + str(ind))
                bias.data.zero_()
                bias.data[input_rnn.hidden_size: 2 * input_rnn.hidden_size] = 1
                bias = eval('input_rnn.bias_hh_l' + str(ind))
                bias.data.zero_()
                bias.data[input_rnn.hidden_size: 2 * input_rnn.hidden_size] = 1
            if input_rnn.bidirectional:
                for ind in range(0, input_rnn.num_layers):
                    bias = eval('input_rnn.bias_ih_l' + str(ind) + '_reverse')
                    bias.data.zero_()
                    bias.data[input_rnn.hidden_size: 2 *
                              input_rnn.hidden_size] = 1
                    bias = eval('input_rnn.bias_hh_l' + str(ind) + '_reverse')
                    bias.data.zero_()
                    bias.data[input_rnn.hidden_size: 2 *
                              input_rnn.hidden_size] = 1

## BiLSTM

In [ ]:
class BiLSTM(nn.Module):

    def __init__(self, vocab_size: int, embed_dim=128, hid_dim=128,
                 loss_fn=nn.NLLLoss(), num_layers=2, tag_size=4,
                 use_dropout=False, dropout=0.1,
                 pre_word_embeds=None,
                 use_subembed=True, subembed_dim=64, subhid_dim=64,
                 use_init=False, init_fns=None, device='cpu'):
        super(BiLSTM, self).__init__()

        self.use_dropout = use_dropout
        if use_dropout is True:
            self.dropout = nn.Dropout(dropout)

        self.embedding = nn.Embedding(vocab_size, embed_dim)
        if pre_word_embeds is not None:
            self.embedding.weight = nn.Parameter(
                torch.FloatTensor(pre_word_embeds))
            self.embedding.weight.requires_grad = False
        elif use_init:
            init_fns.init_embedding(self.embedding)

        self.use_subembed = use_subembed
        if use_subembed:
            self.subembedding = nn.Embedding(vocab_size, subembed_dim)
            if use_init:
                init_fns.init_embedding(self.subembedding)
            self.subnn = nn.GRU(subembed_dim, subhid_dim)
            if use_init:
                init_fns.init_rnn(self.subnn, 'gru')

            embed_dim += subhid_dim
            if use_init:
                init_fns.init_embedding(self.subembedding)

        self.rnn = nn.LSTM(embed_dim, hid_dim,
                           bidirectional=True, num_layers=num_layers)
        if use_init:
            init_fns.init_rnn(self.rnn, 'lstm')

        self.hid2tag = nn.Linear(hid_dim * 2, tag_size)
        if use_init:
            init_fns.init_linear(self.hid2tag)

        self.loss_fn = loss_fn

    def _forward(self, x):
        embeds = self.embedding(x).view(len(x), 1, -1)

        if self.use_subembed:
            _embed = self.subembedding(x)
            sub_out, _ = self.subnn(_embed.view(len(x), 1, -1))
            embeds = torch.cat([embeds, sub_out], dim=2)

        if self.use_dropout:
            self.dropout(embeds)

        lstm_out, _ = self.rnn(embeds)
        if self.use_dropout:
            self.dropout(lstm_out)

        tag_space = self.hid2tag(lstm_out.view(len(x), -1))
        output = F.log_softmax(tag_space, dim=1)
        return output

    def forward(self, x):
        output = self._forward(x)
        tag_seq = torch.argmax(output, dim=1).cpu().numpy()
        return tag_seq

    def loss(self, x, y):
        output = self._forward(x)
        return self.loss_fn(output, y)

## BiLSTM + CRF

In [ ]:
class BiLSTMCRF(nn.Module):

    def __init__(self, vocab_size, embed_dim=10, hid_dim=20, layer_num=2, device='cpu',
                pre_word_embeds=None):
        super(BiLSTMCRF, self).__init__()
        self.embed_dim = embed_dim
        self.hid_dim = hid_dim
        self.vocab_size = vocab_size
        self.tags_size = len(tag_to_ix)
        self.layer_num = layer_num
        self.device = device
        self.word_embeds = nn.Embedding(vocab_size, embed_dim)
        if pre_word_embeds is not None:
            self.word_embeds.weight = nn.Parameter(
                torch.FloatTensor(pre_word_embeds))
            self.word_embeds.weight.requires_grad = False
         
        self.rnn = nn.LSTM(embed_dim, hid_dim // 2,
                           num_layers=self.layer_num, bidirectional=True, dropout=0.4)
        init_rnn(self.rnn, 'lstm')
        self.hid2tag = nn.Linear(hid_dim, self.tags_size)
        init_linear(self.hid2tag)
        self.transitions = nn.Parameter(
            torch.randn(self.tags_size, self.tags_size,device=device))
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

    def init_hidden(self):
        return (nn.init.xavier_uniform_(torch.empty(2 * self.layer_num, 1, self.hid_dim // 2, device=self.device), gain=nn.init.calculate_gain('relu')),
                nn.init.xavier_uniform_(torch.empty(2 * self.layer_num, 1, self.hid_dim // 2, device=self.device), gain=nn.init.calculate_gain('relu')))

    def _forward_alg(self, feats):
        init_alphas = torch.full((1, self.tags_size), -10000., device=self.device)
        init_alphas[0][tag_to_ix[START_TAG]] = 0.
        forward_var = init_alphas
        for feat in feats:
            alphas_t = []
            for next_tag in range(self.tags_size):
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tags_size)
                trans_score = self.transitions[next_tag].view(1, -1)
                next_tag_var = forward_var + trans_score + emit_score
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, x):
        hidden = self.init_hidden()
        embeds = self.word_embeds(x).view(
            len(x), 1, -1)
        lstm_out, _ = self.rnn(embeds, hidden)
        lstm_out = lstm_out.view(len(x), self.hid_dim)
        lstm_feats = self.hid2tag(lstm_out)

        return lstm_feats

    def _score_sentence(self, feats, tags):
        score = torch.zeros(1,device=self.device)
        tags = torch.cat(
            [torch.tensor([tag_to_ix[START_TAG]], dtype=torch.long, device=self.device), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        backpointers = []
        init_vvars = torch.full((1, self.tags_size), -10000.,device=self.device)
        init_vvars[0][tag_to_ix[START_TAG]] = 0
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []
            viterbivars_t = []
            for next_tag in range(self.tags_size):
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)
        terminal_var = forward_var + \
            self.transitions[tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        start = best_path.pop()
        assert start == tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()  # 把从后向前的路径正过来
        return best_path

    def loss(self, x, y):
        feats = self._get_lstm_features(x)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, y)
        return forward_score - gold_score

    def forward(self, x):  # dont confuse this with _forward_alg above.
        lstm_feats = self._get_lstm_features(x)
        tag_seq = self._viterbi_decode(lstm_feats)
        return tag_seq

## Encoder + Att + Decoder + CRF

In [ ]:
class EncoderAttDecoder(nn.Module):

    def __init__(self, vocab_size, in_hdim=128, out_hdim=128, de_hdim: int=128, loss_fn=nn.NLLLoss(),
                 use_crf=True, max_length=1000,
                 bi=2, device='cpu', dropout=0.4, num_layers=2,
                 teacher_forcing_ratio=0.3,pre_word_embeds=None):
        super(EncoderAttDecoder, self).__init__()
        self.bi = bi
        self.device = device
        self.max_length = max_length
        self.loss_fn = loss_fn if use_crf is False else None
        self.num_layers = num_layers
        self.en_in_hdim = in_hdim
        self.en_out_hdim = out_hdim
        self.tags_size = len(tag_to_ix)
        self.de_hdim = de_hdim
        self.teacher_forcing_ratio = teacher_forcing_ratio
        self.use_crf = use_crf
        self.dropout = nn.Dropout(dropout)

        self.en_wrd_embed = nn.Embedding(vocab_size, self.en_in_hdim)
        if pre_word_embeds is not None:
            self.en_wrd_embed.weight = nn.Parameter(
                torch.FloatTensor(pre_word_embeds))
            self.en_wrd_embed.weight.requires_grad = False
            
        #init_embedding(self.en_wrd_embed)
        self.enrnn = nn.GRU(self.en_in_hdim, self.en_out_hdim, num_layers=num_layers,
                            bidirectional=True if bi == 2 else False)
        #init_rnn(self.enrnn, 'GRU')
        self.de_embed = nn.Embedding(self.tags_size, self.de_hdim)
        #init_embedding(self.de_embed)
        self.attn = nn.Linear(self.de_hdim * 2, self.max_length)
        #init_linear(self.attn)
        self.attn_combine = nn.Linear(self.de_hdim * 2, self.de_hdim)
        #init_linear(self.attn_combine)
        self.dernn = nn.GRU(self.de_hdim, self.de_hdim)  # , dropout=0.3)
        #init_rnn(self.dernn, 'GRU')
        self.hid2tag = nn.Linear(self.de_hdim, self.tags_size)
        #init_linear(self.hid2tag)
        self.transitions = nn.Parameter(torch.nn.init.uniform_(
            torch.empty(self.tags_size, self.tags_size,device=self.device)))
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

    def _encoder(self, x):
        eoutputs = torch.zeros(
            self.max_length, self.en_out_hdim * self.bi, device=self.device)
        ehidden = nn.init.xavier_uniform_(
            torch.zeros(self.bi * self.num_layers, 1,
                        self.en_out_hdim, device=self.device),
            gain=nn.init.calculate_gain('relu'))
        for ei in range(x.size(0)):
            eoutput, ehidden = self._encoder_net(
                x[ei], ehidden)
            eoutputs[ei] = eoutput[0, 0]

        return eoutputs, ehidden

    def _encoder_net(self, sentence, hidden):
        embed = self.en_wrd_embed(sentence).view(1, 1, -1)
        #embed = self.dropout(embed)
        output, hidden = self.enrnn(embed, hidden)
        #output = self.dropout(output)
        output = F.log_softmax(output, dim=2)
        return output, hidden

    def _decoder(self, eoutputs, ehidden, y=None):
        dinput = torch.tensor([[tag_to_ix[START_TAG]]], device=self.device)
        if self.bi ==2:
            dhidden = torch.cat(
                [ehidden[0, :, :], ehidden[-1, :, :]], 1).unsqueeze(0)
        else:
            dhidden = ehidden
        doutputs = torch.zeros(
            self.length, self.de_hdim, device=self.device)
        if y is not None:
            for di in range(self.length):
                doutput, dhidden = self._decoder_net(
                    dinput, dhidden, eoutputs)
                dinput = y[di]  # Teacher forcing
                doutputs[di] = doutput[0, 0]
        else:
            for di in range(self.length):
                doutput, decoder_hidden = self._decoder_net(
                    dinput, dhidden, eoutputs)
                topv, topi = doutput.topk(1)
                dinput = topi.squeeze().detach()
                doutputs[di] = doutput[0, 0]
                if dinput.item() == STOP_TAG:
                    break
        return doutputs

    def _decoder_net(self, input, hidden, encoder_outputs):
        embedded = self.de_embed(input).view(1, 1, -1)
        #embedded = self.dropout(embedded)
        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))
        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)
        output = F.relu(output)
        output, hidden = self.dernn(output, hidden)
        #output = self.dropout(output)
        output = F.log_softmax(self.hid2tag(output[0]), dim=1)
        return output, hidden

    def _crf(self, feats):
        return self._viterbi_decode(feats)

    def _forward_alg(self, feats):
        init_alphas = torch.full((1, self.tags_size), -10000.,device=self.device)
        init_alphas[0][tag_to_ix[START_TAG]] = 0.
        forward_var = init_alphas
        # Iterate through the sentence
        for feat in feats:
            alphas_t = []
            for next_tag in range(self.tags_size):
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tags_size)
                trans_score = self.transitions[next_tag].view(1, -1)
                next_tag_var = forward_var + trans_score + emit_score
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _score_sentence(self, feats, tags):
        score = torch.zeros(1,device=self.device)
        tags = torch.cat(
            [torch.tensor([tag_to_ix[START_TAG]], dtype=torch.long, device=self.device), tags])
        for i, feat in enumerate(feats):
            score += self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score += self.transitions[tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _neg_log_likelihood(self, feats, tags):
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def _viterbi_decode(self, feats):
        backpointers = []
        init_vvars = torch.full((1, self.tags_size), -10000.,device=self.device)
        init_vvars[0][tag_to_ix[START_TAG]] = 0
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []
            viterbivars_t = []
            for next_tag in range(self.tags_size):
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)
        terminal_var = forward_var + \
            self.transitions[tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        start = best_path.pop()
        assert start == tag_to_ix[START_TAG]
        best_path.reverse()
        return best_path

    def _get_feat(self, x, y=None, use_tf=False):
        self.length = x.size(0)
        eoutputs, ehidden = self._encoder(x)
        if use_tf and random.random() < self.teacher_forcing_ratio:
            doutputs = self._decoder(eoutputs, ehidden, y)
        else:
            doutputs = self._decoder(eoutputs, ehidden)
        output = self.hid2tag(doutputs)
        return output
    
    def _nll_loss(self, output, y):
        loss = 0
        for i in range(y.size(0)):
            loss += self.loss_fn(output[i].unsqueeze(0), y[i].unsqueeze(0))
        return loss

    def loss(self, x, y):
        output = self._get_feat(x, y, use_tf=True)
        if self.use_crf:
            return self._neg_log_likelihood(output, y)
        else:
            return self._nll_loss(output, y)

    def forward(self, x):
        output = self._get_feat(x)
        if self.use_crf:
            tag_seq = self._crf(output)
        else:
            tag_seq = torch.argmax(output, dim=1).cpu().numpy()
        return tag_seq

# Train and Evaluate

## prepare

In [11]:
trn_X, trn_y, tst, word_to_ix = load()
max_length = max(len(x) for x in trn_X)

max_length

566

In [ ]:
def train(trn_ixs, model, optimizer,device='cpu'):
    avg_loss = 0
    tsize = len(trn_ixs)
    for i in np.random.permutation(trn_ixs): #tqdm(np.random.permutation(trn_ixs)):#
        x = torch.tensor(trn_X[i], dtype=torch.long, device=device)
        y = torch.tensor(trn_y[i], dtype=torch.long, device=device)
        optimizer.zero_grad()
        loss = model.loss(x, y)
        avg_loss += loss.item() / tsize
        loss.backward()
        optimizer.step()
    return model, avg_loss
def evaluate(vld_ixs, model,device='cpu'):
    vld_loss = 0
    vsize = len(vld_ixs)
    y_preds, y_trues = [], []
    for i in np.random.permutation(vld_ixs):#tqdm(np.random.permutation(vld_ixs)):#
        x = torch.tensor(trn_X[i], dtype=torch.long, device=device)
        y = torch.tensor(trn_y[i], dtype=torch.long, device=device)
        y_pred = model(x)
        loss = model.loss(x, y)
        vld_loss += loss.item() / vsize
        y_preds.append(y_pred[:])
        y_trues.append(trn_y[i])
    return vld_loss, y_preds, y_trues

## setting

In [13]:
split_num = 1
epoch_num = 4
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
direction = 2
en_indim = 128
en_outdim = 128
device

device(type='cuda')


## Train Embedding

In [ ]:
ftmodel = fasttext.train_unsupervised(os.path.join(DATAPATH, 'corp.txt'),
                                    model='skipgram',
                                    dim=128,
                                    epoch=10,
                                    ws=5,
                                    minCount=1,
                                    loss='hs',
                                    wordNgrams=2)


In [ ]:
ftmodel.save_model(os.path.join(MODELPATH, 'skipgram.bin'))

## Grid Search BiLSTM

In [ ]:
seed = 10
vld_size = 0.2
edim = 128
hdim = 128
num_layers = 2
tag_to_ix = {"a": 0, "b": 1, "c": 2, "o": 3, }
ix_to_tag = {tp[1]: tp[0] for tp in tag_to_ix.items()}
tagix_to_frq = {0: 0, 1: 0, 2: 0, 3: 0, }
for ix in range(len(trn_X)):
    for t in trn_y[ix]:
        tagix_to_frq[t] += 1
frqs = [tagix_to_frq[t] for t in [0, 1, 2, 3]]
alpha = [sum(frqs) / f for f in frqs]
split_num = 2 # 重复两次实验

for i1, pre_embed_name in enumerate([None, 'cbow']):
    if pre_embed_name is not None:
        pretrianed_embedding = load_pretrained(pre_embed_name, word_to_ix)
    else:
        pretrianed_embedding = None

    for i2, loss_fn in enumerate([FocalLoss(4, alpha=alpha, size_average=False), FocalLoss(4, alpha=alpha, size_average=True), FocalLoss(4, size_average=False), FocalLoss(4, size_average=True),nn.NLLLoss()]):

        for i3, int_fn in enumerate([None, nn.init.normal_, nn.init.uniform_, nn.init.xavier_uniform_, nn.init.xavier_normal_, nn.init.kaiming_uniform_, nn.init.kaiming_normal_]):

            if int_fn is None:
                use_init = False
                init_fns = None
            else:
                use_init = True
                init_fns = InitFns(int_fn, int_fn, int_fn)
            if i3 < 5:
                continue
            for i4, drp in enumerate([None, 0.1, 0.3, 0.5]):
                if drp is None:
                    use_dropout = False
                else:
                    use_dropout = True
                
                for i5, subdim in enumerate([None]):#[None, 32, 64, 128]):
                    if subdim is None:
                        use_subembed = False
                    else:
                        use_subembed = True

                    for i6, lr in enumerate([0.005,]): #0.01, 0.02, 0.05]):
                        no = '{}{}{}{}{}{}'.format(i1, i2, i3, i4, i5, i6)
                        if no in ['000000','000001','000002','000003','000010','000011','000012','000013','000020','000021','000022',
                                 '000300','000301','000302','000303','000310']:
                            continue
                        for s in range(split_num):
                            
                            trn_ixs, vld_ixs = train_test_split(
                                list(range(len(trn_X))), test_size=vld_size,
                                shuffle=True, random_state=seed+s)
                            model = BiLSTM(vocab_size=len(word_to_ix), tag_size=4,
                                       embed_dim=edim, hid_dim=hdim, num_layers=num_layers,
                                       loss_fn=loss_fn,
                                       use_dropout=use_dropout, dropout=drp,
                                       pre_word_embeds=pretrianed_embedding,
                                       use_subembed=use_subembed, subembed_dim=subdim, subhid_dim=subdim,
                                       use_init=use_init, init_fns=init_fns, device=device).to(device)
                            optimizer = optim.Adam(
                                model.parameters(), lr=lr, weight_decay=1e-6)

                            process(trn_ixs, vld_ixs, model, optimizer, device,
                                epoch_num=5,
                                mname='bilstm',
                                no=no)




No 005000 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


## BiLSTM

In [ ]:
seed = 10
use_pre = False
pre_embed_name = 'cbow'
vld_size = 0.2
lr = 0.03
epoch_num = 4
use_alpha = True
use_subembed = False
use_dropout = True
use_init = True
edim = 128
hdim = 128
dropout=0.1
num_layers=2
tag_to_ix = {"a": 0, "b": 1, "c": 2, "o": 3, }
ix_to_tag = {tp[1]: tp[0] for tp in tag_to_ix.items()}
if use_pre:
    pretrianed_embedding = load_pretrained(pre_embed_name, word_to_ix)
else:
    pretrianed_embedding = None
# https://www.zhihu.com/question/67926031
if use_alpha:
    tagix_to_frq = {0: 0, 1: 0, 2: 0, 3: 0, }
    for ix in trn_ixs:
        for t in trn_y[ix]:
            tagix_to_frq[t] += 1
    frqs = [tagix_to_frq[t] for t in [0, 1, 2, 3]]
    alpha = [sum(frqs) / f for f in frqs]
else:
    alpha = None

init_fns = {'rnn': init_rnn,
            'embed': init_embedding,
            'linear': init_linear}

In [ ]:
for sid in range(split_num):
    trn_ixs, vld_ixs = train_test_split(
        list(range(len(trn_X))), test_size=vld_size,
        shuffle=True, random_state=sid)
    print('SPLIT {} --------------------'.format(sid + 1))
    print()
    #loss_fn = FocalLoss(len(tag_to_ix), alpha=alpha, size_average=False)
    loss_fn = nn.NLLLoss()
    model = BiLSTM(vocab_size=len(word_to_ix), tag_size=len(tag_to_ix),
               embed_dim=edim, hid_dim=hdim, num_layers=num_layers,
               loss_fn=loss_fn,
               use_dropout=use_dropout, dropout=dropout,
               pre_word_embeds=pretrianed_embedding,
               use_subembed=use_subembed, subembed_dim=64, subhid_dim=64,
               use_init=use_init, init_fns=init_fns,device=device).to(device)
    optimizer = optim.Adam(
        model.parameters(), lr=0.01, weight_decay=1e-6)
    for epoch in range(epoch_num):
        model.train()
        model, avg_loss = train(trn_ixs, model, optimizer,device)
        model.eval()
        vld_loss, y_preds, y_trues = evaluate(vld_ixs, model,device)
        print('Epoch {}/{} \t avg_loss {:.4f} \t vld_loss {:.4f} \t'.format(
            epoch + 1, epoch_num, avg_loss, vld_loss))
        evalinfo(y_preds, y_trues)
    savemodel(model, 'bilstm_{}_fl.pytorch'.format(sid))

SPLIT 1 --------------------

Epoch 1/4 	 avg_loss 0.2571 	 vld_loss 0.2092 	
TAG a 	 prec 0.6445 	 recl 0.6445 	 f1 0.6445
TAG b 	 prec 0.6483 	 recl 0.6483 	 f1 0.6483
TAG c 	 prec 0.7086 	 recl 0.7086 	 f1 0.7086
AVG prec 0.6671 	 recl 0.6671 	 f1 0.6671
Epoch 2/4 	 avg_loss 0.2127 	 vld_loss 0.2109 	
TAG a 	 prec 0.5232 	 recl 0.5232 	 f1 0.5232
TAG b 	 prec 0.6421 	 recl 0.6421 	 f1 0.6421
TAG c 	 prec 0.7202 	 recl 0.7202 	 f1 0.7202
AVG prec 0.6285 	 recl 0.6285 	 f1 0.6285
Epoch 3/4 	 avg_loss 0.2058 	 vld_loss 0.2099 	
TAG a 	 prec 0.6461 	 recl 0.6461 	 f1 0.6461
TAG b 	 prec 0.6459 	 recl 0.6459 	 f1 0.6459
TAG c 	 prec 0.7014 	 recl 0.7014 	 f1 0.7014
AVG prec 0.6645 	 recl 0.6645 	 f1 0.6645
Epoch 4/4 	 avg_loss 0.2033 	 vld_loss 0.2063 	
TAG a 	 prec 0.5785 	 recl 0.5785 	 f1 0.5785
TAG b 	 prec 0.6154 	 recl 0.6154 	 f1 0.6154
TAG c 	 prec 0.7217 	 recl 0.7217 	 f1 0.7217
AVG prec 0.6385 	 recl 0.6385 	 f1 0.6385


## BiLSTM + CRF

In [ ]:
for sid in range(split_num):
    trn_ixs, vld_ixs = train_test_split(
        list(range(len(trn_X))), test_size=1 / split_num,
        shuffle=True, random_state=sid)
    print('SPLIT {} --------------------'.format(sid + 1))
    model = BiLSTMCRF(len(word_to_ix), embed_dim=200,
                      hid_dim=200, device=device).to(device)
    optimizer = optim.Adam(
        model.parameters(), lr=0.01, weight_decay=1e-6)
    for epoch in range(epoch_num)h:
        model.train()
        model, avg_loss = train(trn_ixs, model, optimizer)
        model.eval()
        vld_loss, y_preds, y_trues = evaluate(vld_ixs, model)
        print('Epoch {}/{} \t avg_loss {:.4f} \t vld_loss {:.4f} \t'.format(
            epoch + 1, epoch_num, avg_loss, vld_loss))
        evalinfo(y_preds, y_trues)
    savemodel(model, 'bilstmcrf_{}.pytorch'.format(sid))

## Encoder + Att + Decoder

In [ ]:
for sid in range(split_num):
    trn_ixs, vld_ixs = train_test_split(
        list(range(len(trn_X))), test_size=1 / split_num,
        shuffle=True, random_state=sid)
    print('SPLIT {} --------------------'.format(sid + 1))
    pre_word_embeds = load_pretrained('cbow', word_to_ix)
    model = EncoderAttDecoder(len(word_to_ix), in_hdim=en_indim, out_hdim=en_outdim,
                              de_hdim=en_outdim * direction, max_length=max_length,
                              bi=direction, dropout=0.3, num_layers=2,
                              teacher_forcing_ratio=0.3, use_crf=False, device=device,
                             pre_word_embeds=pre_word_embeds).to(device)
    optimizer = optim.Adam(
        model.parameters(), lr=0.01, weight_decay=1e-6)
    for epoch in range(epoch_num):
        model.train()
        model, avg_loss = train(trn_ixs, model, optimizer)
        model.eval()
        vld_loss, y_preds, y_trues = evaluate(vld_ixs, model)
        print('Epoch {}/{} \t avg_loss {:.4f} \t vld_loss {:.4f} \t'.format(
            epoch + 1, epoch_num, avg_loss, vld_loss))
        evalinfo(y_preds, y_trues)
        savemodel(model, 'ead_{}_{}.pytorch'.format(sid,epoch))

SPLIT 1 --------------------


Epoch 1/4 	 avg_loss -42295322913.0458 	 vld_loss -115566925728.9789 	
TAG a 	 prec 0.0000 	 recl 0.0000 	 f1 0.0000
TAG b 	 prec 0.0000 	 recl 0.0000 	 f1 0.0000
TAG c 	 prec 0.0000 	 recl 0.0000 	 f1 0.0000
AVG prec 0.0000 	 recl 0.0000 	 f1 0.0000


KeyboardInterrupt: ignored

## Encoder + Att + Decoder + CRF

In [ ]:
for sid in range(split_num):
    trn_ixs, vld_ixs = train_test_split(
        list(range(len(trn_X))), test_size=1 / split_num,
        shuffle=True, random_state=sid)
    print('SPLIT {} --------------------'.format(sid + 1))

    model = EncoderAttDecoder(len(word_to_ix), in_hdim=en_indim, out_hdim=en_outdim,
                              de_hdim=en_outdim * direction, max_length=max_length,
                              bi=direction, dropout=0.3, num_layers=2,
                              teacher_forcing_ratio=0.3, use_crf=True, device=device).to(device)
    optimizer = optim.Adam(
        model.parameters(), lr=0.01, weight_decay=1e-6)
    for epoch in range(epoch_num):
        model.train()
        model, avg_loss = train(trn_ixs, model, optimizer)
        model.eval()
        vld_loss, y_preds, y_trues = evaluate(vld_ixs, model)
        print('Epoch {}/{} \t avg_loss {:.4f} \t vld_loss {:.4f} \t'.format(
            epoch + 1, epoch_num, avg_loss, vld_loss))
        evalinfo(y_preds, y_trues)
        savemodel(model, 'eadcrf_{}_{}.pytorch'.format(sid,epoch))